We developed a function to determine if any of the descriptions in the corpus are similar to an external description and then print the comparison values for each description. In order to determine the similarities, We used the cosine similarity method. The cosine similarity measures the similarity between two documents. If the value is closer to 1 then those two documents are similar. In my function, We loop through each description in the corpus, compare them and calculate a value for each of them. Once the values are printed, We find the maximum value and return the description text.


$$ cos(x,y) = \frac{a \cdot b}{||a|| ||b||} $$

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import cross_val_score
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
df=pd.read_csv("df_search_repeat.csv")

In [3]:
df.head()

,developer_response,Count
0,سعيدين بخدمتك وشكرا لتقيمك نتطلع لخدمتك مستقبلا,3
1,مرحبا عذرا منك علي ذلك ! نود الحصول علي تفاصي...,4
2,يسرنا سماع ذلك ونتمني ان نكون عند حسن ظنك دائم...,2
3,مرحبا محمد عذرا منك علي ذلك ! نود الحصول علي ...,3
4,مرحبا سعدنا بخدمتك ! نقدر لك وقتك لمشاركه تجر...,229


In [4]:
df.shape

(9190, 2)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import ArabicStemmer

stemmer = ArabicStemmer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

stem_vectorizer = CountVectorizer(analyzer=stemmed_words)

print(stem_vectorizer.fit_transform(['أهلاً بك، سنعمل على توصيل ملاحظاتك للقسم المختص ونتمنى أن نراك قريباً مجدداً!']))
print(stem_vectorizer.get_feature_names())

doc_vectors = stem_vectorizer.fit_transform(df['developer_response'])
print(doc_vectors.shape)
#print(vectorizer.get_feature_names())

  (0, 1)	1
  (0, 2)	1
  (0, 4)	1
  (0, 5)	1
  (0, 3)	1
  (0, 10)	1
  (0, 7)	1
  (0, 9)	1
  (0, 11)	1
  (0, 0)	1
  (0, 12)	1
  (0, 6)	1
  (0, 8)	1
['أن', 'اهل', 'بك', 'توصيل', 'سنعمل', 'على', 'قريب', 'قسم', 'مجدد', 'مختص', 'ملاحظ', 'نتمنى', 'نرا']
(9190, 3479)


# Classifcation

In [150]:
df_test = pd.read_csv("df_search_not_repeat.csv")

In [151]:
df_test.shape

(43566, 2)

In [152]:
df_test.head()

,developer_response,Count
0,We sincerely thank you for taking the time to ...,1
1,مرحبا يسعدنا سماع ذلك وهدفنا الاول ارضاؤكم شكرا,1
2,مرحبا اشرف يسرنا سماع ذلك ونتمني ان نكون عند ح...,1
3,مرحبا عمار يسرنا سماع ذلك ونتمني ان نكون عند ح...,1
4,مرحبا توم يسرنا سماع ذلك ونتمني ان نكون عند حس...,1


In [153]:
# create a list text
df_text_not_repeat = list(df_test['developer_response'])
print (len(df_text_not_repeat))

43566


In [154]:
#initial list for first run 
#indexLabel = [None] * len (df_test)
#simList    = [None] * len (df_test)

indexLabel = list(df_test['label'])
simList = list(df_test['sim_value'])

In [155]:
def comp_description(query, results_number=20):
    results=[]
    q_vector = stem_vectorizer.transform([query])    
    results.append(cosine_similarity(q_vector, doc_vectors.toarray()))
    f=0
    elem_list=[]
    
    for i in results[:1]:
        for elem in i[0]:
                #print("Review",f, "Similarity: ", elem)
                elem_list.append(elem)
                f+=1    
    
    #print("Similarity: ", max(elem_list))
    sim = max(elem_list)
    if sum(elem_list) / len(elem_list) == 0.0:        
        sim = 0.0    
    return sim
           

In [135]:
#df_text_not_repeat[100:102]

In [176]:
from time import time  # To time our operations
t = time()

start = 2000

for index , item in enumerate(df_text_not_repeat [start :]):
    print(index + start , item)
    
    sim = comp_description (item)  
    
    simList[index + start ] =   round (sim, 2)
    #print ("{} - Similarity = {}".format(index,  round (sim, 2)))
    
    if  round (sim, 2)  > 0.95:            
        indexLabel [index + start ] = "Generic"

    if index == 1000 :
        break

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

2000 مرحبا فايز يسعدنا معرفه انك تستمتع باستخدام تطبيق بوتيم !  ونقدر لك تخصيص بعض الوقت لكتابه مثل هذه التعليقات الرائعه نامل ان تكون تجربتك ايجابيه دائما مع بوتيم !  خالص التحيات !
2001 مرحبا Aly نقدر لك الوقت الذي قضيته في كتابه هذه التعليقات الرائعه لقد ادخلت البهجه والسرور علي قلوبنا !
2002 مرحبا Mohmmed يسعدنا معرفه انك تستمتع باستخدام تطبيق بوتيم !  ونقدر لك تخصيص بعض الوقت لكتابه مثل هذه التعليقات الرائعه نامل ان تكون تجربتك ايجابيه دائما مع بوتيم !  خالص التحيات !
2003 مرحبا AlShebli يسعدنا معرفه انك تستمتع باستخدام تطبيق بوتيم !  ونقدر لك تخصيص بعض الوقت لكتابه مثل هذه التعليقات الرائعه نامل ان تكون تجربتك ايجابيه دائما مع بوتيم !  خالص التحيات !
2004 مرحبا Shahed نشكرك علي اتاحه دقائق قليله من وقتك الثمين لمشاركه تجربتك معنا نتطلع لرسم الابتسامه علي وجهك دائما !
2005 مرحبا Kareem لقد اسعدتنا كلماتك الطيبه كثيرا شكرا لمشاركه رايك معنا ومع عائله بوتيم
2006 Hi Mohamady, Thank you so much for taking the time to leave us this amazing review This made our day !
2007 مرحبا Mahmoud 

2054 هلا مارجو شكرا علي كلماتك الطيبه يسعدنا انضمامك الي عائله بوتيم !  نتطلع لخدمتك مره اخري قريبا !
2055 مرحبا روبي لقد اسعدتنا كلماتك الطيبه كثيرا شكرا لمشاركه رايك معنا ومع عائله بوتيم
2056 مرحبا عبد الله ردوكم وتعليقاتكم تساعدنا علي تطوير وتحسين خدماتنا يسعدنا سماع انك استمتعت بتجربه رائعه مع بوتيم ونتطلع للترحيب بكم مره اخري قريبا
2057 مرحبا هيما نقدر لك الوقت الذي قضيته في كتابه هذه التعليقات الرائعه لقد ادخلت البهجه والسرور علي قلوبنا !
2058 هلا صهيب شكرا علي كلماتك الطيبه يسعدنا انضمامك الي عائله بوتيم !  نتطلع لخدمتك مره اخري قريبا !
2059 مرحبا نحن معروفون عاده بجوده مكالماتنا الاستثنائيه لذا ناسف اننا لم نرق الي هذه التوقعات هذه المره اذا لم يكن لديك مانع من مناقشه هذا الامر بتوسع اكبر يرجي مراسلتنا عبر البريد الالكتروني reviews@botimme اننا نقدر بشده اتاحه الفرصه لنا لتصحيح الامور والعمل علي استعادتك ضمن عملائنا المميزين مره اخري
2060 مرحبا حاتم نقدر لك الوقت الذي قضيته في كتابه هذه التعليقات الرائعه لقد ادخلت البهجه والسرور علي قلوبنا !
2061 شكرا علي مراجعاتك الرائعه عبدال

2106 Hi Waleed, Thank you so much for taking the time to leave us this amazing review This made our day !
2107 مرحبا Ghaith شكرا جزيلا علي تعليقك نسعي دائما للتاكد من ان جميع مستخدمينا يتمتعون بتجارب رائعه من الواضح اننا اخفقنا في انجاز ذلك هذه المره لذلك نعتذر نود الحصول علي مزيد من التعليقات منك حتي نتمكن من تحسين تطبيقنا يرجي ارسال ملاحظاتك الينا علي reviews@botimme
2108 مرحبا هبه يسعدنا معرفه انك تستمتع باستخدام تطبيق بوتيم !  ونقدر لك تخصيص بعض الوقت لكتابه مثل هذه التعليقات الرائعه نامل ان تكون تجربتك ايجابيه دائما مع بوتيم !  خالص التحيات !
2109 Hi Yousef, Thank you so much for taking the time to leave us this amazing review This made our day !
2110 مرحبا Fahad لقد اسعدتنا كلماتك الطيبه كثيرا شكرا لمشاركه رايك معنا ومع عائله بوتيم
2111 مرحبا Taha نشكرك علي اتاحه دقائق قليله من وقتك الثمين لمشاركه تجربتك معنا نتطلع لرسم الابتسامه علي وجهك دائما !
2112 مرحبا السلام امان لقد اسعدتنا كلماتك الطيبه كثيرا شكرا لمشاركه رايك معنا ومع عائله بوتيم
2113 مرحبا عبدالباسط شكرا جزيلا علي تعليق

2164 مرحبا فهد نشكرك علي اتاحه دقائق قليله من وقتك الثمين لمشاركه تجربتك معنا نتطلع لرسم الابتسامه علي وجهك دائما !
2165 مرحبا شريف شكرا لك علي ترك تعليقاتك نسعي جاهدين لتقديم تجربه ممتعه لكل لعميل وناسف حقا لاننا لم نرق لتوقعاتك هذه المره نود حل هذه المشكله في اسرع وقت ممكن وايجاد حلا يناسبك لها لذا يرجي ارسال رساله الينا مباشره علي البريد الالكتروني reviews@botimme لكي نتمكن من مناقشه الامر بتوسع اكثر
2166 مرحبا نواف نقدر لك الوقت الذي قضيته في كتابه هذه التعليقات الرائعه لقد ادخلت البهجه والسرور علي قلوبنا !
2167 مرحبا Rana يسعدنا معرفه انك تستمتع باستخدام تطبيق بوتيم !  ونقدر لك تخصيص بعض الوقت لكتابه مثل هذه التعليقات الرائعه نامل ان تكون تجربتك ايجابيه دائما مع بوتيم !  خالص التحيات !
2168 Hi Banan, We are so grateful for your kind words Thanks for sharing your review with us and the community
2169 هلا هشام شكرا علي كلماتك الطيبه يسعدنا انضمامك الي عائله بوتيم !  نتطلع لخدمتك مره اخري قريبا !
2170 مرحبا Rezk يسعدنا معرفه انك تستمتع باستخدام تطبيق بوتيم !  ونقدر لك تخصيص بعض الوقت

2213 مرحبا Lamia نشكرك علي اتاحه دقائق قليله من وقتك الثمين لمشاركه تجربتك معنا نتطلع لرسم الابتسامه علي وجهك دائما !
2214 مرحبا Saher نشكرك علي اتاحه دقائق قليله من وقتك الثمين لمشاركه تجربتك معنا نتطلع لرسم الابتسامه علي وجهك دائما !
2215 مرحبا عبد الحليم نشكرك علي اتاحه دقائق قليله من وقتك الثمين لمشاركه تجربتك معنا نتطلع لرسم الابتسامه علي وجهك دائما !
2216 مرحبا هشام نشكرك علي اتاحه دقائق قليله من وقتك الثمين لمشاركه تجربتك معنا نتطلع لرسم الابتسامه علي وجهك دائما !
2217 مرحبا رجاء يسعدنا معرفه انك تستمتع باستخدام تطبيق بوتيم !  ونقدر لك تخصيص بعض الوقت لكتابه مثل هذه التعليقات الرائعه نامل ان تكون تجربتك ايجابيه دائما مع بوتيم !  خالص التحيات !
2218 مرحبا Ahmed شكرا جزيلا علي تعليقك نسعي دائما للتاكد من ان جميع مستخدمينا يتمتعون بتجارب رائعه من الواضح اننا اخفقنا في انجاز ذلك هذه المره لذلك نعتذر نود الحصول علي مزيد من التعليقات منك حتي نتمكن من تحسين تطبيقنا يرجي ارسال ملاحظاتك الينا علي reviews@botimme
2219 مرحبا Sadek نقدر لك الوقت الذي قضيته في كتابه هذه التعليقات الرائعه لقد

...

2746 مرحبا طه يسرنا سماع ان تجربتك مع بوتيم كانت ايجابيه !  نحن نقدر ارائك ويسرنا معرفه المزيد عن تجربتك مع بوتيم
2747 مرحبا نجيب شكرا علي تعليقك الرائع نحن نبذل قصاري جهدنا لنكون عند مستوي توقعات عملائنا ويسعدنا ان نسمع ان خدماتنا لاقت اعجابكم نتطلع لخدمتك في المستقبل خالص التحيات !
2748 مرحبا Mohammed لقد اسعدتنا كلماتك الطيبه كثيرا شكرا لمشاركه رايك معنا ومع عائله بوتيم
2749 مرحبا Ahmad شكرا علي تعليقك الرائع نحن نبذل قصاري جهدنا لنكون عند مستوي توقعات عملائنا ويسعدنا ان نسمع ان خدماتنا لاقت اعجابكم نتطلع لخدمتك في المستقبل خالص التحيات !
2750 مرحبا تاج السر يسعدنا معرفه انك تستمتع باستخدام تطبيق بوتيم !  ونقدر لك تخصيص بعض الوقت لكتابه مثل هذه التعليقات الرائعه نامل ان تكون تجربتك ايجابيه دائما مع بوتيم !  خالص التحيات !
2751 مرحبا Eman يسعدنا معرفه انك تستمتع باستخدام تطبيق بوتيم !  ونقدر لك تخصيص بعض الوقت لكتابه مثل هذه التعليقات الرائعه نامل ان تكون تجربتك ايجابيه دائما مع بوتيم !  خالص التحيات !
2752 مرحبا Jamal شكرا علي تعليقك الرائع نحن نبذل قصاري جهدنا لنكون عند مستوي توقعا

2798 مرحبا طيبه يسرنا سماع ان تجربتك مع بوتيم كانت ايجابيه !  نحن نقدر ارائك ويسرنا معرفه المزيد عن تجربتك مع بوتيم
2799 مرحبا Mohammed يسرنا سماع ان تجربتك مع بوتيم كانت ايجابيه !  نحن نقدر ارائك ويسرنا معرفه المزيد عن تجربتك مع بوتيم
2800 هلا Bashaer شكرا علي كلماتك الطيبه يسعدنا انضمامك الي عائله بوتيم !  نتطلع لخدمتك مره اخري قريبا !
2801 هلا Eman شكرا علي كلماتك الطيبه يسعدنا انضمامك الي عائله بوتيم !  نتطلع لخدمتك مره اخري قريبا !
2802 مرحبا Sameh يسعدنا معرفه انك تستمتع باستخدام تطبيق بوتيم !  ونقدر لك تخصيص بعض الوقت لكتابه مثل هذه التعليقات الرائعه نامل ان تكون تجربتك ايجابيه دائما مع بوتيم !  خالص التحيات !
2803 مرحبا Mayada شكرا جزيلا علي تعليقك نسعي دائما للتاكد من ان جميع مستخدمينا يتمتعون بتجارب رائعه من الواضح اننا اخفقنا في انجاز ذلك هذه المره لذلك نعتذر نود الحصول علي مزيد من التعليقات منك حتي نتمكن من تحسين تطبيقنا يرجي ارسال ملاحظاتك الينا علي reviews@botimme
2804 مرحبا Attia نشكرك علي اتاحه دقائق قليله من وقتك الثمين لمشاركه تجربتك معنا نتطلع لرسم الابتسامه علي وجهك

2853 مرحبا نجيم نقدر ولائكم واخلاصك لنا ونشكرك علي مشاركه تجربتكم معنا !  نتطلع للعمل سويا مره اخري !
2854 مرحبا مياسين شكرا علي تعليقاتك القيمه نود معرفه مزيدا من التفاصيل بشان تجربتك لكي نتمكن من الاستفاده من تعليقاتك في تقديم تجربه افضل لك في المره القادمه يرجي التواصل مع علي reviews@botimme لتقديم اي تعليقات او اقتراحات اخري ترغب في مشاركتها معنا
2855 مرحبا اديب لقد اسعدتنا كلماتك الطيبه كثيرا شكرا لمشاركه رايك معنا ومع عائله بوتيم
2856 مرحبا ريحان يسرنا سماع ان تجربتك مع بوتيم كانت ايجابيه !  نحن نقدر ارائك ويسرنا معرفه المزيد عن تجربتك مع بوتيم
2857 مرحبا مازن شكرا علي تعليقاتك القيمه نود معرفه مزيدا من التفاصيل بشان تجربتك لكي نتمكن من الاستفاده من تعليقاتك في تقديم تجربه افضل لك في المره القادمه يرجي التواصل مع علي reviews@botimme لتقديم اي تعليقات او اقتراحات اخري ترغب في مشاركتها معنا
2858 مرحبا شيرين يسرنا سماع ان تجربتك مع بوتيم كانت ايجابيه !  نحن نقدر ارائك ويسرنا معرفه المزيد عن تجربتك مع بوتيم
2859 مرحبا حنان يسعدنا معرفه انك تستمتع باستخدام تطبيق بوتيم !  ونقدر لك تخصي

2909 هلا كاميليا شكرا علي كلماتك الطيبه يسعدنا انضمامك الي عائله بوتيم !  نتطلع لخدمتك مره اخري قريبا !
2910 مرحبا رضا نقدر ولائكم واخلاصك لنا ونشكرك علي مشاركه تجربتكم معنا !  نتطلع للعمل سويا مره اخري !
2911 مرحبا اسلام نشكرك علي الوقت الذي قضيته في كتابه تعليقك وناسف لان تجربتك في بوتيم لم تتوافق تماما مع توقعاتك نحن ناخذ جميع المراجعات علي محمل الجد ونستعين بها في اطار جهودنا المستمره لتحسين الجوده !  يرجي عدم التردد في التواصل معنا علي reviews@botimme لتقديم اي تعليقات او مخاوف او اقتراحات اخري ترغب في مشاركتها معنا
2912 مرحبا عبدالرزاق شكرا علي تعليقاتك القيمه نود معرفه مزيدا من التفاصيل بشان تجربتك لكي نتمكن من الاستفاده من تعليقاتك في تقديم تجربه افضل لك في المره القادمه يرجي التواصل مع علي reviews@botimme لتقديم اي تعليقات او اقتراحات اخري ترغب في مشاركتها معنا
2913 Hi Moataz, Thank you for sharing your kind words with us We are thrilled to have you as part of the BOTIM family !  We look forward to serving you again soon !
2914 مرحبا كرم لقد اسعدتنا كلماتك الطيبه كثيرا شكرا لم

2956 مرحبا شحته يسرنا سماع ان تجربتك مع بوتيم كانت ايجابيه !  نحن نقدر ارائك ويسرنا معرفه المزيد عن تجربتك مع بوتيم
2957 مرحبا مها نقدر ولائكم واخلاصك لنا ونشكرك علي مشاركه تجربتكم معنا !  نتطلع للعمل سويا مره اخري !
2958 مرحبا سامي شكرا علي تعليقاتك القيمه نود معرفه مزيدا من التفاصيل بشان تجربتك لكي نتمكن من الاستفاده من تعليقاتك في تقديم تجربه افضل لك في المره القادمه يرجي التواصل مع علي reviews@botimme لتقديم اي تعليقات او اقتراحات اخري ترغب في مشاركتها معنا
2959 مرحبا سلمان يسرنا سماع ان تجربتك مع بوتيم كانت ايجابيه !  نحن نقدر ارائك ويسرنا معرفه المزيد عن تجربتك مع بوتيم
2960 مرحبا جهاد يسرنا سماع ان تجربتك مع بوتيم كانت ايجابيه !  نحن نقدر ارائك ويسرنا معرفه المزيد عن تجربتك مع بوتيم
2961 مرحبا جبران نقدر ولائكم واخلاصك لنا ونشكرك علي مشاركه تجربتكم معنا !  نتطلع للعمل سويا مره اخري !
2962 مرحبا تحسين نقدر ولائكم واخلاصك لنا ونشكرك علي مشاركه تجربتكم معنا !  نتطلع للعمل سويا مره اخري !
2963 مرحبا باسل  شكرا علي تعليقاتك اننا نسعي جاهدين للتميز لذا ناسف لسماع ان حظيت بتجربه متوسطه

In [177]:
#simList [100:200]

In [178]:
#indexLabel[100:200]

In [179]:
#search for frequencies sentnecte and count to map
mp = dict()
def count(arr, n):    
    # Traverse through array elements
    # and count frequencies
    for i in range(n):
        if arr[i] in mp.keys():
            mp[arr[i]] += 1
        else:
            mp[arr[i]] = 1

n = len(indexLabel) 
print (n)
count(indexLabel, n)

43566


In [180]:
mp

{None: 41833, 'Generic': 1733}

In [181]:
df_test['label'] = indexLabel
df_test['sim_value'] =  simList

In [182]:
#save data to csv file after cleaning data
df_test.to_csv("df_test.csv", encoding='utf-8-sig', index=False , header=True)